<a href="https://colab.research.google.com/github/geekevgin/-Python/blob/main/UBelEvSer_NLP_HW_14.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import json
import numpy as np
import re
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments, AutoModelForCausalLM

In [2]:
!pip install -q transformers

In [4]:
model_name = "sberbank-ai/rugpt3small_based_on_gpt2"

In [5]:
from google.colab import files
uploaded = files.upload()

Saving dataset.jsonl to dataset (2).jsonl


In [6]:
df = pd.read_json('dataset (2).jsonl', lines=True).set_index('id') 

In [7]:
df

,date,rating,text
id,,,
1,2004-08-30 11:24:00+00:00,22010.0,"<Ares> ppdv, все юниксы очень дружелюбны.. они..."
2,2004-08-30 11:25:00+00:00,25105.0,<томатик_рад> а ты не чувствуешь красоту мира?...
3,2004-08-30 11:27:00+00:00,7192.0,"<Дор> ""мышка, почему у тебя такие большие глаз..."
4,2004-08-30 11:28:00+00:00,29169.0,"<PPDV[os2]> ""Мальчики, вы что больные, бегать ..."
5,2004-08-30 11:26:00+00:00,7140.0,<Ohtori_Akio> мы - как разработчики - живём с ...
...,...,...,...
463644,2020-11-26 06:12:00+00:00,698.0,"xxx: угадайте не гугля, что такое жопиздан!\ny..."
463645,2020-11-26 06:12:00+00:00,816.0,"xxx:\nПосетила шальная мысль заняться собой, ж..."
463646,2020-11-26 06:46:00+00:00,88.0,#всебожьяроса\nxxx: Судьба айтишников вообще н...


In [8]:
data = []
with open('dataset.jsonl', 'r') as json_file:
    json_list = list(json_file)
for json_str in json_list:
    data.append(json.loads(json_str)["text"])

In [9]:
data[:2]

['<Ares> ppdv, все юниксы очень дружелюбны.. они просто очень разборчивы в друзьях ;)',
 '<томатик_рад> а ты не чувствуешь красоту мира?\n<fox> честно говоря, я сейчас чувствую только отсутствие http.\n<томатик_рад> не туда смотришь, глянь вокруг!\n<fox> как я гляну, если http не работает? :/']

In [10]:
len(data)

81497

In [11]:
data = data[:5000]

In [12]:
def clear_text(text):
    clr_text = re.sub(r"<.*?>", " ", text).lower()
    clr_text = summary = re.sub(r"\s", " ", clr_text)
    return clr_text

In [13]:
df["clear_text"] = df["text"].apply(lambda x: clear_text(x))
df.head()

,date,rating,text,clear_text
id,,,,
1,2004-08-30 11:24:00+00:00,22010.0,"<Ares> ppdv, все юниксы очень дружелюбны.. они...","ppdv, все юниксы очень дружелюбны.. они прос..."
2,2004-08-30 11:25:00+00:00,25105.0,<томатик_рад> а ты не чувствуешь красоту мира?...,а ты не чувствуешь красоту мира? честно го...
3,2004-08-30 11:27:00+00:00,7192.0,"<Дор> ""мышка, почему у тебя такие большие глаз...","""мышка, почему у тебя такие большие глаза?"" ..."
4,2004-08-30 11:28:00+00:00,29169.0,"<PPDV[os2]> ""Мальчики, вы что больные, бегать ...","""мальчики, вы что больные, бегать в палату к..."
5,2004-08-30 11:26:00+00:00,7140.0,<Ohtori_Akio> мы - как разработчики - живём с ...,мы - как разработчики - живём с субейзом под...


In [14]:
text = df.loc[:, 'clear_text']
text

id
1           ppdv, все юниксы очень дружелюбны.. они прос...
2           а ты не чувствуешь красоту мира?   честно го...
3           "мышка, почему у тебя такие большие глаза?" ...
4           "мальчики, вы что больные, бегать в палату к...
5           мы - как разработчики - живём с субейзом под...
                                ...                        
463644    xxx: угадайте не гугля, что такое жопиздан! yy...
463645    xxx: посетила шальная мысль заняться собой, жи...
463646    #всебожьяроса xxx: судьба айтишников вообще не...
463647    прибывшие на место правоохранители установили,...
463648    ххх: вот ребята из английской фирмы tvr (давны...
Name: clear_text, Length: 81497, dtype: object

In [15]:
def build_text_files(data_json, dest_path):
    f = open(dest_path, 'w')
    data = ''
    for texts in data_json:
        summary = str(texts).strip()
        data += summary + "  "
    f.write(data)

In [16]:
train, test = train_test_split(text, test_size=0.15)

In [17]:
build_text_files(train,'train_dataset.txt')
build_text_files(test,'test_dataset.txt')

In [18]:
print("Train dataset length: "+ str(len(train)))
print("Test dataset length: "+ str(len(test)))

Train dataset length: 69272
Test dataset length: 12225


In [19]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

train_path = 'train_dataset.txt'
test_path = 'test_dataset.txt'

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [20]:
def load_dataset(train_path, test_path, tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)

    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=128)

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset, test_dataset, data_collator

train_dataset, test_dataset, data_collator = load_dataset(train_path, test_path, tokenizer)

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [21]:
model = AutoModelForCausalLM.from_pretrained(model_name)

In [22]:
!pip install --upgrade accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [23]:
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

Found existing installation: transformers 4.29.2
Uninstalling transformers-4.29.2:
  Successfully uninstalled transformers-4.29.2
Found existing installation: accelerate 0.19.0
Uninstalling accelerate-0.19.0:
  Successfully uninstalled accelerate-0.19.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached transformers-4.29.2-py3-none-any.whl (7.1 MB)
  Using cached accelerate-0.19.0-py3-none-any.whl (219 kB)


In [22]:
exit()

In [31]:
training_args = TrainingArguments(

    "phrase",
    evaluation_strategy = "epoch",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=5,
    learning_rate=1e-5,
    weight_decay=0.01,
    save_strategy='no',
    report_to='none',
    
    )


In [32]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

In [33]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,3.837800,3.860089
2,3.804900,3.843111
3,3.730000,3.839674
4,3.678800,3.838556
5,3.640800,3.840968


TrainOutput(global_step=45060, training_loss=3.7182438660345234, metrics={'train_runtime': 8693.4073, 'train_samples_per_second': 20.731, 'train_steps_per_second': 5.183, 'total_flos': 1.17728391168e+16, 'train_loss': 3.7182438660345234, 'epoch': 5.0})

In [27]:
def generate_text(prefix):
    tokens = tokenizer(prefix, return_tensors='pt').to('cuda')
    size = tokens['input_ids'].shape[1]

    output = model.generate(
        **tokens, 
        #end_token=end_token_id,
        do_sample=False,
        max_length=size+50, 
        early_stopping=True,
        length_penalty=2.0,
        repetition_penalty=8., 
        temperature=0.5,
        num_beams=3,
        no_repeat_ngram_size=5
    )

    decoded = tokenizer.decode(output[0])
    result = decoded[len(prefix):]
    return prefix + result

In [35]:
print(generate_text("привет, как дела?!"))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


привет, как дела?!"  xxx: у нас на работе в туалете висит табличка с надписью "не курить", а рядом надпись "в туалет не ходить". так вот. сегодня утром я решил сходить к унитазу покурить и обнаружил там записку от неизвестного мне


In [36]:
print(generate_text("На улице тепло, я пойду гулять"))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


На улице тепло, я пойду гулять.  xxx: у нас на работе есть один чувак с фамилией алексеевич yyy: и что он делает? xxx: ну как бы это сказать... живет в общаге со своей девушкой-студенткой


In [37]:
print(generate_text("Как выучить математику за две недели?"))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Как выучить математику за две недели?  xxx: у нас на работе есть один сотрудник, который постоянно что-то пишет. и вот однажды он написал в своей книге "как стать миллионером" (на самом деле это не книга, а рассказ о том, как заработать миллион


In [38]:
print(generate_text("Где купить картофель?"))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Где купить картофель?  xxx: у нас в городе есть магазин, где можно купить картошку. yyy: ну и как он называется? xxx: "картофельная ферма". zzz: а что это за ферма такая?
